In [2]:
import pandas as pd
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import getpass
import re

### Spotify API connection  

In [3]:
c_id = getpass.getpass()
c_se = getpass.getpass()

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se, requests_timeout=1000))

### Database connexion

In [5]:
pw = "password1999"
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string, pool_pre_ping = False)

### Merging recommandations tables -whithout audio features-

In [7]:
genres = ["acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "anime", "black-metal", "bluegrass", "blues", "brazil", "breakbeat", "british", "cantopop", "chicago-house", "children", "chill", "classical", "club", "comedy", "country", "dance", "dancehall", "death-metal", "deep-house", "detroit-techno", "disco", "drum-and-bass", "dub", "dubstep", "edm", "electro", "electronic", "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel", "goth", "grindcore", "groove", "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal", "hip-hop", "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance", "j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal", "metalcore", "minimal-techno",  "mpb", "new-age", "opera", "pagode", "party", "piano", "pop", "pop-film", "power-pop", "progressive-house", "psych-rock", "punk", "punk-rock", "r-n-b", "reggae", "reggaeton","rock", "rock-n-roll", "rockabilly", "romance", "sad", "salsa", "samba", "sertanejo", "show-tunes", "singer-songwriter", "ska", "sleep", "songwriter", "soul", "spanish", "study", "swedish", "synth-pop", "tango", "techno", "trance", "trip-hop", "turkish", "world-music"]

In [8]:
reco_tables = ["df_{}_reco".format(genre.replace("-","")) for genre in genres ]

In [9]:
def merging_reco_tables() :
    pattern = "(?<=_)[^_]+(?=_)"

    df_reco = pd.read_sql("SELECT * FROM `spotify`.`{}`".format(reco_tables[0]), engine)
    df_reco.insert(2,"track_genre",re.search(pattern, reco_tables[0]).group(0))

    for table_name in reco_tables[1:] :
        df_table = pd.read_sql("SELECT * FROM `spotify`.`{}`".format(table_name), engine) #import table in df
        df_table.insert(2,"track_genre",re.search(pattern, table_name).group(0)) #add the column "track_genre" to df
        df_reco = pd.concat([df_reco,df_table])
    
    return df_reco

In [12]:
df_merger_reco = merging_reco_tables()

In [16]:
df_merger_reco.drop_duplicates(inplace = True, keep = "first")

In [17]:
df_merger_reco.to_sql("df_all_rec_no_audio", connection_string, schema='spotify', if_exists='replace', index = False)

455876

### Getting audio features

In [29]:
def get_audio_features(df_without_audio_features) :
    
    list_of_dict = []

    for i in range(0, df_without_audio_features.shape[0], 100):
        time.sleep(1)
        sub_list_track_id = list(df_without_audio_features["track_id"][i:i + 100])
        sub_list_audio_featues = sp.audio_features(sub_list_track_id) # list of dict : {'danceability': 0.365, 'energy': 0.76, 'key': 4, 'loudness': -6.311, 'mode': 1, 'speechiness': 0.0395, 'acousticness': 0.00328, 'instrumentalness': 6.5e-06, 'liveness': 0.936, 'valence': 0.132, 'tempo': 77.024, 'type': 'audio_features', 'id': '4dd1w41SxD7OAcAVpzUBJL', 'uri': 'spotify:track:4dd1w41SxD7OAcAVpzUBJL', 'track_href': 'https://api.spotify.com/v1/tracks/4dd1w41SxD7OAcAVpzUBJL', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4dd1w41SxD7OAcAVpzUBJL', 'duration_ms': 391947, 'time_signature': 4}
        list_of_dict.extend(sub_list_audio_featues) # extend() avoid to have a list of list but just a list
    
    result =  pd.DataFrame.from_dict(list_of_dict)


    # Merging
    merge_df = df_without_audio_features.merge(result, left_on = "track_id", right_on = "id")
    merge_df = merge_df.drop("id", axis= 1)

    return merge_df

In [60]:
def get_audio_features(df_without_audio_features) :
    
    list_of_dict = []
    total_df = pd.DataFrame()

    for i in range(0, df_without_audio_features.shape[0], 100):
        time.sleep(1)
        sub_list_track_id = list(df_without_audio_features["track_id"][i:i + 100])
        sub_list_audio_featues = sp.audio_features(sub_list_track_id) # list of dict : {'danceability': 0.365, 'energy': 0.76, 'key': 4, 'loudness': -6.311, 'mode': 1, 'speechiness': 0.0395, 'acousticness': 0.00328, 'instrumentalness': 6.5e-06, 'liveness': 0.936, 'valence': 0.132, 'tempo': 77.024, 'type': 'audio_features', 'id': '4dd1w41SxD7OAcAVpzUBJL', 'uri': 'spotify:track:4dd1w41SxD7OAcAVpzUBJL', 'track_href': 'https://api.spotify.com/v1/tracks/4dd1w41SxD7OAcAVpzUBJL', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4dd1w41SxD7OAcAVpzUBJL', 'duration_ms': 391947, 'time_signature': 4}
        sub_list_audio_featues = [d for d in sub_list_audio_featues if d]

        sub_df = pd.DataFrame(sub_list_audio_featues)
        total_df = pd.concat([total_df, sub_df]) 

    # Merging
    merge_df = df_without_audio_features.merge(total_df, left_on = "track_id", right_on = "id")
    merge_df = merge_df.drop("id", axis= 1)

    return merge_df

In [68]:
selected_rows = df_merger_reco[::9]

In [69]:
df_reco_audio_9 = get_audio_features(selected_rows)

In [70]:
df_reco_audio_9.to_sql("df_reco_audio_9", connection_string, schema='spotify', if_exists='replace', index = False)

63840

### Merging complet dataframe

In [6]:
df_complet = pd.read_sql("SELECT * FROM spotify.df_complet", engine)
df_reco_audio_10 = pd.read_sql("SELECT * FROM spotify.df_reco_audio_10", engine)
df_reco_audio_9 = pd.read_sql("SELECT * FROM spotify.df_reco_audio_9", engine)

df_complet = pd.concat([df_complet,df_reco_audio_10,df_reco_audio_9])

In [8]:
df_complet.shape

(207963, 29)

In [9]:
df_complet.drop_duplicates(inplace = True)
df_complet.shape

(178039, 29)

In [11]:
df_complet.to_sql("df_complet", connection_string, schema='spotify', if_exists='replace', index = False)

178039